Nota generada a partir de [liga](https://www.dropbox.com/s/yjijtfuky3s5dfz/2.5.Compute_Unified_Device_Architecture.pdf?dl=0)

**Notas para contenedor de docker:**

Comando de docker para ejecución de la nota de forma local:

nota: cambiar `<ruta a mi directorio>` por la ruta de directorio que se desea mapear a `/datos` dentro del contenedor de docker.

```
docker run --gpus all --rm -v $(pwd):/datos --name jupyterlab_nvidia_cuda_c_container -p 8888:8888 -d palmoreck/jupyterlab_nvidia_cuda_c:1.1.0
```

password para jupyterlab: `qwerty`

Detener el contenedor de docker:

```
docker stop jupyterlab_nvidia_cuda_c_container
```

Documentación de la imagen de docker `palmoreck/jupyterlab_nvidia_cuda_c:1.1.0` en [liga](https://github.com/palmoreck/dockerfiles/tree/master/jupyterlab/nvidia/cuda_c).

---

**Nota: si se desean ejecutar los ejemplos que se presentan a continuación, es necesario tener una tarjeta gráfica NVIDIA.**

# CUDA C y generalidades de CUDA y GPU

## CUDA C

Consiste en extensiones al lenguaje C y en una *runtime library*. Ver [2.3.CUDA](https://github.com/ITAM-DS/analisis-numerico-computo-cientifico/blob/master/temas/II.computo_paralelo/2.3.CUDA.ipynb) para más información.

### Kernel

* En CUDA C se define una función que se ejecuta en el device y que se le nombra **kernel**. El *kernel* inicia con la sintaxis:

```
__global__ void mifun(int param){
...
}

```

y siempre es tipo `void` (no hay `return`).

* El llamado al *kernel* se realiza desde el host y con una sintaxis en la que se define el número de threads y bloques que serán utilizados para la ejecución del kernel. La sintaxis que se utiliza es con `<<< >>>` y en la primera entrada se coloca el número de bloques y en la segunda entrada el número de *threads*:


```
__global__ void mifun(int param){
...
}

int main(){
    int par;
    mifun<<<N,5>>> (par); //N bloques de 5 threads
}

```

## Ejemplos

### 1) Programa de hello world

In [1]:
%%file hello_world.cu
#include<stdio.h>
__global__ void func(void){
}
int main(void){
    func<<<1,1>>>(); //1 bloque de 1 thread
    printf("Hello world!\n");
return 0;
}

Writing hello_world.cu


Compilación:

In [11]:
%%bash
nvcc --compiler-options -Wall hello_world.cu -o hello_world.out

Ejecución:

In [4]:
%%bash
./hello_world.out

Hello world!


**Comentarios:**

* La función `main` se ejecuta en la CPU.

* La función `func` es un *kernel* y es ejecutada por *threads* en el *device* (GPU), también llamados *CUDA threads*. Obsérvese que la función `func` inicia con `__global__` para diferenciarla de `main`.

* El *kernel* sólo puede tener un `return` tipo *void*: `__global__ void func` por lo que el *kernel* debe regresar sus resultados a través de sus argumentos.
 

### 2) Programa de hello world 2

In [5]:
%%file hello_world_2.cu
#include<stdio.h>
__global__ void func(void){
    printf("Hello world del bloque %d del thread %d!\n", blockIdx.x, threadIdx.x);
}
int main(void){
    func<<<2,3>>>(); //2 bloques de 3 threads cada uno
    cudaDeviceSynchronize();
    printf("Hola del cpu thread\n");
    return 0;
}


Writing hello_world_2.cu


In [12]:
%%bash 
nvcc --compiler-options -Wall hello_world_2.cu -o hello_world_2.out

In [7]:
%%bash
./hello_world_2.out

Hello world del bloque 1 del thread 0!
Hello world del bloque 1 del thread 1!
Hello world del bloque 1 del thread 2!
Hello world del bloque 0 del thread 0!
Hello world del bloque 0 del thread 1!
Hello world del bloque 0 del thread 2!
Hola del cpu thread


**Comentarios:**

* La extensión del archivo debe ser `.cu` aunque esto puede modificarse al compilar con `nvcc`: 

`$nvcc -x cu hello_world.c -o hello_world.out`

* El llamado a la ejecución del kernel se realizó en el *host* y se lanzaron $2$ bloques (primera posición), cada uno con $3$ *threads*.

* Se utiliza la función [cudaDeviceSynchronize](https://docs.nvidia.com/cuda/cuda-runtime-api/group__CUDART__DEVICE.html#group__CUDART__DEVICE_1g10e20b05a95f638a4071a655503df25d) para que el *cpu-thread* espere la finalización de la ejecución del kernel.

* Los *CUDA threads* son divididos en bloques y todos los bloques se encuentran en un *grid*. En el lanzamiento del *kernel*  se debe especificar al hardware cuántos bloques tendrá nuestro *grid* y cuántos *threads* estarán en cada bloque. Las variables `blockIdx` y `threadIdx` hacen referencia a los **id**'s que tienen los bloques y los threads. El *id* del bloque dentro del *grid* y el *id* del thread dentro del bloque. La parte `.x` de `blockIdx.x` y `threadIdx.x` refiere a la primera coordenada del bloque y del *thread* en el *grid* y en el bloque respectivamente. 

* La elección del número de bloques en un grid o el número de *threads* en un bloque no corresponde a alguna disposición del hardware, esto es, si se lanza un kernel con `<<< 1, 3 >>>` no implica que la GPU tenga un bloque o 3 *threads*.

* En una GPU podemos definir un *grid* de bloques y un bloque de *threads* utilizando un tipo de dato `dim3` el cual también es parte de CUDA C:

In [8]:
%%file hello_world_3.cu
#include<stdio.h>
__global__ void func(void){
    printf("Hello world del bloque %d del thread %d!\n", blockIdx.x, threadIdx.x);
}
int main(void){
    dim3 dimGrid(2,1); //2 bloques en el grid
    dim3 dimBlock(3,1,1); //3 threads por bloque
    func<<<dimGrid,dimBlock>>>(); 
    cudaDeviceSynchronize();
    printf("Hola del cpu thread\n");
    return 0;
}

Writing hello_world_3.cu


In [13]:
%%bash 
nvcc --compiler-options -Wall hello_world_3.cu -o hello_world_3.out

In [10]:
%%bash
./hello_world_3.out

Hello world del bloque 1 del thread 0!
Hello world del bloque 1 del thread 1!
Hello world del bloque 1 del thread 2!
Hello world del bloque 0 del thread 0!
Hello world del bloque 0 del thread 1!
Hello world del bloque 0 del thread 2!
Hola del cpu thread


### 3) Programa de suma vectorial

In [ ]:
%%file suma_vectorial.cu
#include<stdio.h>
#define N 10
__global__ void suma_vect(int *a, int *b, int *c){
    int tid = blockIdx.x;
    if(tid<N) //suposición N menor al número máximo de bloques que se pueden lanzar
    c[tid] = a[tid]+b[tid];
}
int main(void){
    int a[N], b[N],c[N];
    int *device_a, *device_b, *device_c;
    int i;
    //alojando en device
    cudaMalloc((void **)&device_a, sizeof(int)*N);
    cudaMalloc((void **)&device_b, sizeof(int)*N);
    cudaMalloc((void **)&device_c, sizeof(int)*N);
    //llenando los arreglos:
    for(i=0;i<N;i++){
        a[i]=i;
        b[i]=i*i;
    }
    //copiamos arreglos a, b a la GPU
    cudaMemcpy(device_a,a,N*sizeof(int), cudaMemcpyHostToDevice);
    cudaMemcpy(device_b,b,N*sizeof(int), cudaMemcpyHostToDevice);
    //mandamos a llamar a suma_vect:
    suma_vect<<<N,1>>>(device_a,device_b,device_c); //N bloques de 1 thread
    cudaDeviceSynchronize();
    //copia del resultado al arreglo c:
    cudaMemcpy(c,device_c,N*sizeof(int),cudaMemcpyDeviceToHost);
    for(i=0;i<N;i++)
        printf("%d+%d = %d\n",a[i],b[i],c[i]);
    cudaFree(device_a);
    cudaFree(device_b);
    cudaFree(device_c);
    return 0;
}

In [ ]:
%%bash

nvcc suma_vectorial.cu -o suma_vectorial.out

### Comentarios respecto al programa de `suma_vectorial.cu` anterior

* 

### Regla compuesta del rectángulo

In [ ]:
nvcc --compiler-options -Wall Rcf.cu -o Rcf.out

In [ ]:
#include<stdio.h>
#include <thrust/reduce.h>
#include <thrust/execution_policy.h>

__global__ void Rcf(double *data, double a, double h_hat, int n, double *sum ) {
double x=0.0;

if(threadIdx.x<=n-1){
x=a+(threadIdx.x+1/2.0)*h_hat;
data[threadIdx.x]=std::exp(-std::pow(x,2));
}
    *sum = thrust::reduce(thrust::device, data , data + n, (double)0, thrust::plus<double>());
}


int main(int argc, char *argv[]){
    double suma=0.0;
    double *d_data;
    double *d_suma;
    double a=0.0, b=1.0;
    double h_hat;
    int n=1e3; //número de subintervalos
    double objetivo=0.7468241328124271;
    cudaMalloc((void **)&d_data,sizeof(double)*n);
    cudaMalloc((void**)&d_suma,sizeof(double));
    h_hat=(b-a)/n;
    Rcf<<<1,n>>>(d_data, a,h_hat,n,d_suma); //1 bloque de n threads
    cudaDeviceSynchronize();
    cudaMemcpy(&suma, d_suma, sizeof(double), cudaMemcpyDeviceToHost);
    suma=h_hat*suma;
    cudaFree(d_data) ;
    cudaFree(d_suma) ;
    printf("Integral de %f a %f = %1.15e\n", a,b,suma);
    printf("Error relativo de la solución: %1.15e\n", fabs(suma-objetivo)/fabs(objetivo));
    return 0;
}

Integral de 0.000000 a 1.000000 = 7.468241634690490e-01

Error relativo de la solución: 4.104931878976858e-08

**Referencias**

1. N. Matloff, Parallel Computing for Data Science. With Examples in R, C++ and CUDA, 2014.

2. [CUDA](https://github.com/ITAM-DS/analisis-numerico-computo-cientifico/tree/master/C/extensiones_a_C/CUDA)

3. [2.3.CUDA](https://github.com/ITAM-DS/analisis-numerico-computo-cientifico/blob/master/temas/II.computo_paralelo/2.3.CUDA.ipynb)